# Modélisation de la prédiction du prix d'une maison à New York

In [33]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, r2_score

import warnings
warnings.filterwarnings("ignore")

# Ajoute le chemin du dossier parent à sys.path
notebook_dir = os.path.abspath('..')  # Dossier parent de "notebook/"
sys.path.append(notebook_dir)

from fonctions.fct_importation import *

In [27]:
data = pd.read_csv(r"C:\Users\najim\Documents\Master 2\MLOps\projet\data\data_clean\data_preprocess.csv")
data

,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,BOROUGH
0,Multi-family home for sale,1100000,6,3,2837,Bronx
1,Multi-family home for sale,925000,5,2,2750,Bronx
2,Multi-family home for sale,700000,6,2,3053,Bronx
3,Autre_Type,119000,3,2,2184,Bronx
4,Multi-family home for sale,600000,2,2,2184,Bronx
...,...,...,...,...,...,...
4791,Co-op for sale,280000,2,1,1050,Bronx
4792,Co-op for sale,375000,2,1,2184,Bronx
4793,Co-op for sale,167999,1,1,2184,Bronx
4794,Co-op for sale,309000,2,1,2184,Bronx


# Modélisation

In [28]:
# Définir une fonction pour calculer RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [36]:
def data_split(df : pd.DataFrame):
    
    # Définir les caractéristiques et la cible
    X = data.drop(columns=['PRICE'])  # Caractéristiques (features)
    y = data['PRICE']                 # Cible (target)

    # Séparer les colonnes numériques et catégoriques
    numeric_features = ['BEDS', 'BATH', 'PROPERTYSQFT']
    categorical_features = ['TYPE', 'BOROUGH']
    
    return X, y, numeric_features, categorical_features

X, y, num_features, catg_features = data_split(df= data)

In [37]:
def preprocess_data(features : pd.DataFrame, target : pd.Series, numeric_features : list, categorical_features : list):
    
    print("Construction d'un preprocessor pour transformer les données")
    # Construire un transformer pour traiter les colonnes
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),   # Standardisation des colonnes numériques
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Encodage OneHot pour les colonnes catégoriques
        ]
    )
    
    display(preprocessor)
    
    return preprocessor

transformer = preprocess_data(features = X, target = y, numeric_features = num_features, categorical_features = catg_features)

Construction d'un preprocessor pour transformer les données


ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['BEDS', 'BATH', 'PROPERTYSQFT']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['TYPE', 'BOROUGH'])])

In [40]:
def modelisation(features : pd.DataFrame, target : pd.Series, preprocessor = transformer):
    
    # Séparer les données en entraînement et test
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # Définir le scorer pour GridSearchCV
    scoring = {
        'RMSE': make_scorer(rmse, greater_is_better=False),
        'R2': make_scorer(r2_score)
    }
    
    # Définir les modèles
    models = {
        "Random Forest": RandomForestRegressor(random_state=42),
        "Neural Network": MLPRegressor(random_state=42, max_iter=500),
        "Linear Regression": LinearRegression()
    }
    
    # Validation croisée et affichage des résultats
    results = {}
    for name, model in models.items():
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
    
            # Validation croisée uniquement sur les données d'entraînement
        scores = cross_val_score(pipeline, X_train, y_train, scoring=make_scorer(rmse))
        mean_rmse = -np.mean(scores)  # Moyenne des scores (RMSE négatif pour compatibilité scikit-learn)
        
        # Ajuster le modèle sur l'ensemble d'entraînement
        pipeline.fit(X_train, y_train)
        
        # Prédire sur les données de test
        y_pred = pipeline.predict(X_test)
        test_rmse = rmse(y_test, y_pred)
        test_r2 = r2_score(y_test, y_pred)
        
        # Stocker les résultats
        results[name] = {
            "RMSE (train cross-val)": mean_rmse,
            "RMSE (test)": test_rmse,
            "R2 (test)": test_r2
        }

    # Afficher les résultats
    print("Résultats des Modèles :")
    for model, metrics in results.items():
        print(f"Modèle : {model}")
        print(f"  RMSE (train cross-val) : {metrics['RMSE (train cross-val)']:.2f}")
        print(f"  RMSE (test) : {metrics['RMSE (test)']:.2f}")
        print(f"  R2 (test) : {metrics['R2 (test)']:.2f}")
        print("-" * 30)

modelisation(features = X, target = y, preprocessor = transformer)

Résultats des Modèles :
Modèle : Random Forest
  RMSE (train cross-val) : -2657428.69
  RMSE (test) : 2172279.21
  R2 (test) : 0.50
------------------------------
Modèle : Neural Network
  RMSE (train cross-val) : -4667029.19
  RMSE (test) : 3448537.37
  R2 (test) : -0.26
------------------------------
Modèle : Linear Regression
  RMSE (train cross-val) : -3434549.51
  RMSE (test) : 2557392.27
  R2 (test) : 0.31
------------------------------


In [41]:
def optimisation_hyperparametres(features : pd.DataFrame, target : pd.Series, preprocessor = transformer):
    print("*"*50)
    print("Optimisation des hyperparamètres du RandomForest et du Réseau de Neurone")
    print("*"*50)
    print("Division des données en train et test")
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    print("Optimisation du RandomForest en cours ...")
    # Grille de recherche pour Random Forest
    rf_params = {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [10, 20, None]
    }

    rf_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(random_state=42))
    ])

    rf_grid = GridSearchCV(rf_pipeline, rf_params, scoring='neg_mean_squared_error', n_jobs=-1)
    rf_grid.fit(X_train, y_train)
    print("Optimisation du RandomForest terminée.")
    print("\n")
    print("Optimisation du Réseau de neurone en cours ...")
    # Grille de recherche pour Réseau de Neurones
    mlp_params = {
        'model__hidden_layer_sizes': [(32,), (64, 64), (128,)],
        'model__alpha': [0.0001, 0.001, 0.01],  # Regularization term
        'model__learning_rate_init': [0.001, 0.01]
    }

    mlp_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', MLPRegressor(random_state=42, max_iter=500))
    ])

    mlp_grid = GridSearchCV(mlp_pipeline, mlp_params, scoring='neg_mean_squared_error', n_jobs=-1)
    mlp_grid.fit(X_train, y_train)
    print("Optimisation du Réseau de neurone terminée.")
    print("*"*50)
    
    # Meilleurs hyperparamètres et performances
    print("\nRésultats de l'optimisation des hyperparamètres :")
    print(f"Random Forest - Meilleurs paramètres : {rf_grid.best_params_}")
    print(f"Random Forest - Meilleur RMSE (train cross-val) : {-rf_grid.best_score_:.2f}")

    print(f"Neural Network - Meilleurs paramètres : {mlp_grid.best_params_}")
    print(f"Neural Network - Meilleur RMSE (train cross-val) : {-mlp_grid.best_score_:.2f}")

optimisation_hyperparametres(features = X, target = y, preprocessor = transformer)

**************************************************
Optimisation des hyperparamètres du RandomForest et du Réseau de Neurone
**************************************************
Division des données en train et test
Optimisation du RandomForest en cours ...
Optimisation du RandomForest terminée.


Optimisation du Réseau de neurone en cours ...
Optimisation du Réseau de neurone terminée.
**************************************************

Résultats de l'optimisation des hyperparamètres :
Random Forest - Meilleurs paramètres : {'model__max_depth': 10, 'model__n_estimators': 100}
Random Forest - Meilleur RMSE (train cross-val) : 7007646927862.62
Neural Network - Meilleurs paramètres : {'model__alpha': 0.0001, 'model__hidden_layer_sizes': (64, 64), 'model__learning_rate_init': 0.01}
Neural Network - Meilleur RMSE (train cross-val) : 9848579767187.98
